In [ ]:
# Import required libraries
import pandas as pd
import dash
# import dash_html_components as html
from dash import html
# import dash_core_components as dcc
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                options=[
                                        {'label': 'All Sites', 'value': 'ALL'},
                                        {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                        {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                        # {'label': 'CCAFS SLC-40', 'value': 'SLC40'}, # this format for value did not allow the graph plot as that is not the name in the data frame, so I changed it
                                        {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                        {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}
                                        ],
                                        value='ALL',
                                        placeholder='Select a Launch Site here',
                                        searchable=True                                 
                                ),                                
                                html.Br(),
                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                min=0,
                                max=10000,
                                step=1000,
                                marks={
                                        0: '0 KG',
                                        2500: '2500 KG',
                                        5000: '5000 KG',
                                        7500: '7500 KG',
                                        10000: '10000 KG'
                                        },
                                value=[min_payload, max_payload]
                                ),
                                
                                                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
        [Output(component_id='success-pie-chart', component_property='figure')], # I got an error when I put a single Output in [] as it expected a list of outputs. For this to work, then I also have to put it in the 
        [Input(component_id='site-dropdown', component_property='value')] # it was not a problem for input as the input received is like the first input in a list of inputs
)

def get_pie_chart(site):

        if site=='ALL':
                # all_df = spacex_df.groupby('Launch Site')['class'].sum().reset_index() # taking only class will restrict it to only the pie_fig. Finally this did not matter as I created a second app.callback as directed in the question
                # all_df = spacex_df[spacex_df['class']==1].groupby('Launch Site')['class'].count().reset_index()
                
                
                all_df = spacex_df.groupby('Launch Site').sum().reset_index()                 
                pie_fig = px.pie(all_df, values='class', names='Launch Site', title='Total Success Launches By Site')
                
        else:
                specific_df = spacex_df[spacex_df['Launch Site']==site].groupby('class').count().reset_index()
                pie_fig = px.pie(specific_df, values='Launch Site', names='class', title='Total Success Launches for site {}'.format(site))

        return [pie_fig]            # remove [] if it is not in the output at callback

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
        Output(component_id='success-payload-scatter-chart', component_property='figure'),
        [Input(component_id='payload-slider', component_property='value'),
        Input(component_id='site-dropdown', component_property='value')]
)
def get_scatter_chart(load, site):
        if site=='ALL':
                # Obtains a DF with all the Launch Site
                all_df = spacex_df

                # Set a filter for payloads above the lower limit
                lower_limit_filter = all_df['Payload Mass (kg)']>=load[0] 
                
                # Set a filter for payloads below the upper limit
                upper_limit_filter = all_df['Payload Mass (kg)']<=load[1]

                # Set a limit that combines both outputs from the slider
                load_filter = lower_limit_filter & upper_limit_filter # using bitwise.and which is same as &

                all_df = all_df[load_filter]
                scatter_fig = px.scatter(all_df, x='Payload Mass (kg)', y='class', color='Booster Version Category', title='Correlation between Payload and Success for all Sites')
        
        else:
                # Obtains a DF with Launch Site as only those with the value in site
                specific_df = spacex_df[spacex_df['Launch Site']==site]

                # Set a filter for payloads above the lower limit
                lower_limit_filter = specific_df['Payload Mass (kg)']>=load[0]

                # Set a filter for payloads below the upper limit
                upper_limit_filter = specific_df['Payload Mass (kg)']<=load[1]

                # Set a limit that combines both outputs from the slider
                load_filter = lower_limit_filter & upper_limit_filter

                specific_df = specific_df[load_filter]
                scatter_fig = px.scatter(specific_df, x='Payload Mass (kg)', y='class', color='Booster Version Category', title='Correlation between Payload and Success for all Sites')
                
        
        return scatter_fig


        


# Run the app
if __name__ == '__main__':
    app.run_server()
